In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

fn="../prices.txt"
df=pd.read_csv(fn, sep='\s+', header=None, index_col=None)
prices = (df.values).T

nInst ,T = 50,750

## Singlular instrument

In [ ]:
inst = 42
instrPrice = prices[inst]
for i in range(1,len(instrPrice)):
    x_current = instrPrice[i]
    x_prev = instrPrice[:i]
    diff = x_current-x_prev
    tdiff = list(reversed(range(1,len(x_prev)+1)))
    gradient = diff/tdiff
    
    x_next = gradient+x_current
    x_next = np.round(x_next,2)
    # round down a order of magnitude due to error bars
    print(x_current,np.round(x_next.mean(),1))

Probability distribution of next price

In [ ]:
from scipy.stats import gaussian_kde

def softmax(x):
    """Compute softmax values for each set of scores in x."""
    x = np.asarray(x)
    exps = np.exp(x - np.max(x))  # Subtract max for numerical stability
    return exps / np.sum(exps)


density_mat = []
pmax = instrPrice.max()
pmin = instrPrice.min()
padding = (pmax - pmin) * 0.1
price_grid = np.linspace(pmin - padding, pmax + padding, 500)
for i in range(1,len(instrPrice)):
    x_current = instrPrice[i]
    x_prev = instrPrice[:i]
    diff = x_current-x_prev
    tdiff = list(reversed(range(1,len(x_prev)+1)))
    gradient = diff/tdiff
    
    x_next = gradient+x_current
    x_next = np.round(x_next,2)

    scaled = np.round(x_next * 100).astype(int)
    
    # Softmax PMF
    min_val = scaled.min()
    max_val = scaled.max()
    full_range = np.arange(min_val, max_val + 1)/100
    counts = np.bincount(scaled)[min_val:]
    prob = softmax(counts)
    
    if i == 1:
        continue
    # Gaussian KDE
    kde = gaussian_kde(x_next)
    density = kde(price_grid)
    y = density/np.sum(density)
    density_mat.append(y)

density_mat = np.array(density_mat).T
plt.figure(figsize=(20,10))
plt.plot(instrPrice)
plt.imshow(
    density_mat,
    aspect='auto',
    origin='lower',
    extent=[1, 751, instrPrice.min(), instrPrice.max()],
    cmap='viridis'
)
plt.show()

## Full histogram display of gradient targets

In [ ]:
pmin, pmax = np.min(instrPrice), np.max(instrPrice)
padding = (pmax - pmin) * 0.1
fine_price_grid = np.linspace(pmin - padding, pmax + padding, 200)
n = len(fine_price_grid)

heatmap = []
for i in range(1,len(instrPrice)):
    x_current = instrPrice[i]
    x_prev = instrPrice[:i]
    diff = x_current-x_prev
    tdiff = list(reversed(range(1,len(x_prev)+1)))
    gradient = diff/tdiff
    x_next = gradient+x_current
    
    x_next = np.round(x_next,decimals=2)
    unique_x, counts = np.unique(x_next, return_counts=True)
    frequency_dist = np.interp(fine_price_grid,unique_x,counts,left=0,right=0)
    frequency_dist /= np.sum(frequency_dist)
    heatmap.append(frequency_dist)

heatmap = np.array(heatmap).T

plt.imshow(heatmap,
           aspect='auto',
           origin='lower',
           extent=[1.5, 751.5,fine_price_grid[0], fine_price_grid[-1]],
           cmap='viridis')
plt.colorbar(label='Probability Density')
plt.xlabel('Price')
plt.ylabel('Time Step')
plt.title('Distribution of Next Price Over Time')
plt.grid(False) # Grid can be distracting on heatmaps
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def estimate_drift_diffusion(x, dt, num_bins=50):
    x = np.asarray(x)
    dx = x[1:] - x[:-1]
    x_mid = x[:-1]

    bins = np.linspace(np.min(x), np.max(x), num_bins + 1)
    bin_centers = 0.5 * (bins[1:] + bins[:-1])

    drift = np.zeros(num_bins)
    diffusion = np.zeros(num_bins)
    counts = np.zeros(num_bins)

    for i in range(len(dx)):
        bin_idx = np.searchsorted(bins, x_mid[i]) - 1
        if 0 <= bin_idx < num_bins:
            drift[bin_idx] += dx[i]
            diffusion[bin_idx] += dx[i]**2
            counts[bin_idx] += 1

    # Normalize
    valid = counts > 0
    drift[valid] /= counts[valid] * dt
    diffusion[valid] /= 2 * counts[valid] * dt

    return bin_centers[valid], drift[valid], diffusion[valid]

# Example usage
# x = your time series
# dt = time step between samples
# x = np.random.normal(size=10000)  # e.g., Brownian motion
# x_vals, D1, D2 = estimate_drift_diffusion(x, dt=0.01)

x_vals, D1, D2 = estimate_drift_diffusion(prices[42,:],dt=1)
plt.plot(x_vals, D1, label='Drift D1')
plt.plot(x_vals, D2, label='Diffusion D2')
plt.legend()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

# --- Function to estimate drift and diffusion ---
def estimate_drift_diffusion(x, dt, num_bins=50):
    x = np.asarray(x)
    dx = x[1:] - x[:-1]
    x_mid = x[:-1]

    bins = np.linspace(np.min(x), np.max(x), num_bins + 1)
    bin_centers = 0.5 * (bins[1:] + bins[:-1])

    drift = np.zeros(num_bins)
    diffusion = np.zeros(num_bins)
    counts = np.zeros(num_bins)

    for i in range(len(dx)):
        bin_idx = np.searchsorted(bins, x_mid[i]) - 1
        if 0 <= bin_idx < num_bins:
            drift[bin_idx] += dx[i]
            diffusion[bin_idx] += dx[i] ** 2
            counts[bin_idx] += 1

    valid = counts > 0
    drift[valid] /= counts[valid] * dt
    diffusion[valid] /= 2 * counts[valid] * dt

    return bin_centers[valid], drift[valid], diffusion[valid]

# --- Sliding window analysis and heatmap construction ---
def sliding_fpe_heatmap(price, dt, window_size=200, step_size=10, num_bins=50, price_grid_size=200):
    times = []
    price_grid = np.linspace(np.min(price), np.max(price), price_grid_size)
    heatmap = []

    for t_start in range(0, len(price) - window_size, step_size):
        x_window = price[t_start:t_start + window_size]
        x_vals, D1, D2 = estimate_drift_diffusion(x_window, dt, num_bins)

        # Construct PDF over price grid using Gaussian from D1 and D2
        pdf = np.zeros_like(price_grid)
        for mu, sigma2 in zip(D1, D2):
            sigma = np.sqrt(np.maximum(sigma2, 1e-8))
            pdf += norm.pdf(price_grid, loc=mu, scale=sigma)

        pdf /= np.max(pdf)  # Normalize for color intensity
        heatmap.append(pdf)
        times.append(t_start + window_size // 2)

    heatmap = np.array(heatmap).T  # transpose for image: rows=price, cols=time
    return np.array(times), price_grid, heatmap

# --- Example usage ---
price = instrPrice
dt = 1.0
times, price_vals, heat = sliding_fpe_heatmap(price, dt)

# --- Plotting ---
def plot_fpe_heatmap(times, price_vals, heatmap):
    plt.figure(figsize=(12, 6))
    plt.imshow(heatmap, aspect='auto', origin='lower',
               extent=[times[0], times[-1], price_vals[0], price_vals[-1]],
               cmap='inferno')
    plt.xlabel('Time')
    plt.ylabel('Price')
    plt.title('Fokker–Planck PDF Heatmap (Sliding Window)')
    plt.colorbar(label='PDF Intensity')
    plt.show()

# Example:
plot_fpe_heatmap(times, price_vals, heat)
